In [1]:
import sqlite3
import pandas as pd
import numpy as np
from API_Key import key, username, password
import requests
import time
import psycopg2 as pg2
import json

In [2]:
def inputting_to_db(username, password, query):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute(query)
    
    conn.commit()
    conn.close()

In [3]:
# Adding data to tables created 
def adding_to_db(username, password, query, data):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.executemany(query, data)
    
    conn.commit()
    conn.close()

# Creating tables based on Riot Games API Structure

In [4]:
query1 = ('''CREATE TABLE summoner(

                leagueId VARCHAR,
                queueType VARCHAR,
                tier TEXT,
                rank TEXT,
                summonerId VARCHAR,
                summonerName TEXT,
                leaguepoints INTEGER,
                wins INTEGER,
                losses INTEGER,
                veteran BOOLEAN,
                inactive BOOLEAN,
                freshBlood BOOLEAN,
                hotStreak BOOLEAN) ''')

In [5]:
query2 = ('''CREATE TABLE userids(

                id TEXT,
                accountId TEXT,
                puuid TEXT PRIMARY KEY,
                name TEXT,
                profileIconId INTEGER,
                revisionDate BIG INT,
                summonerLevel INTEGER) ''')

In [6]:
query3 = ("""CREATE TABLE matchids(
             puuid text PRIMARY KEY,
             match text)""")

In [21]:
query7 = ('''CREATE TABLE matches(
                match TEXT,
                game_datetime BIGINT,
                game_length FLOAT,
                game_version VARCHAR,
                gold_left INT,
                last_round INT,
                level INT,
                placement INT,
                players_eliminated INT,
                puuid TEXT,
                time_eliminated FLOAT,
                total_damage INT,
                traits TEXT,
                units TEXT) ''')

In [7]:
inputting_to_db(username, password, query1)

In [8]:
inputting_to_db(username, password, query2)

In [9]:
inputting_to_db(username, password, query3)

In [24]:
inputting_to_db(username, password, query7)

# Setting up loop to obtain all ranked TFT players in Diamond and below

In [10]:
def player_collector(key):
    leagues = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
    divisions = ['I', 'II', 'III', 'IV']
    summoner = []
    for league in leagues:
        for division in divisions:
            response = requests.get(
            "https://na1.api.riotgames.com/tft/league/v1/entries/{}/{}?api_key={}".format(league, division, key))
            for user in response.json():
                summoner.append(tuple(user.values()))
    return summoner

In [11]:
players = player_collector(key)

In [12]:
len(players)

4738

# Updating summoner table 

In [13]:
query4 = 'INSERT INTO summoner VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [14]:
adding_to_db(username, password, query4, players)

In [10]:
query5 = 'INSERT INTO userids VALUES (%s,%s,%s,%s,%s,%s,%s)'

In [5]:
def puuid_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT summonerName FROM summoner")
    players = c.fetchall()
    # Getting a list of all summonernames to loop through
    users = [player[0] for player in players]
    summoner = []
    for player in users:
        response = requests.get(
        "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{}?api_key={}".format(player, key))
        if response.status_code == 200:
            summoner.append(tuple(response.json().values()))
        elif response.status_code == 429:
            print(len(summoner))
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return summoner;

In [6]:
puuid = puuid_collector(key, username, password)

100
rate limit exceeded
waiting 2 minutes
200
rate limit exceeded
waiting 2 minutes
300
rate limit exceeded
waiting 2 minutes
400
rate limit exceeded
waiting 2 minutes
500
rate limit exceeded
waiting 2 minutes
600
rate limit exceeded
waiting 2 minutes
699
rate limit exceeded
waiting 2 minutes
799
rate limit exceeded
waiting 2 minutes
899
rate limit exceeded
waiting 2 minutes
999
rate limit exceeded
waiting 2 minutes
1099
rate limit exceeded
waiting 2 minutes
1199
rate limit exceeded
waiting 2 minutes
1299
rate limit exceeded
waiting 2 minutes
1399
rate limit exceeded
waiting 2 minutes
1499
rate limit exceeded
waiting 2 minutes
1599
rate limit exceeded
waiting 2 minutes
1699
rate limit exceeded
waiting 2 minutes
1799
rate limit exceeded
waiting 2 minutes
1898
rate limit exceeded
waiting 2 minutes
1998
rate limit exceeded
waiting 2 minutes
2097
rate limit exceeded
waiting 2 minutes
2197
rate limit exceeded
waiting 2 minutes
2297
rate limit exceeded
waiting 2 minutes
2397
rate limit excee

In [13]:
adding_to_db(username, password, query5, puuid)

In [36]:
def matchid_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT puuid FROM userids")
    players = c.fetchall()
    # Getting a list of all puuids to loop through
    users = [player[0] for player in players]
    games = []
    for player in users:
        response = requests.get(
            "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{}/ids?count=30&api_key={}".format(player, key))
        if response.status_code == 200:
            for matches in response.json():
                games.append((player, matches))
        elif response.status_code == 429:
            print(len(games), games[-2:])
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return games;

In [37]:
match_histories = matchid_collector(key, username, password)

2803 [('yLC44_Jkngmkw2vL8PLtSx83hFwXSJIKrB-xnftl86c8hyI0fY3Tc7Y1j9oc3iGQcsSPB1Njf3slYQ', 'NA1_3742278015'), ('yLC44_Jkngmkw2vL8PLtSx83hFwXSJIKrB-xnftl86c8hyI0fY3Tc7Y1j9oc3iGQcsSPB1Njf3slYQ', 'NA1_3742218814')]
rate limit exceeded
waiting 2 minutes
5531 [('eL4CeoiAa3lsAkERVsE_7Yyh4m5VIIsBgz0bR8HYol-ry-zDYYwNRKCqZ0JKnBiOlnImd2UXfcEXDw', 'NA1_3739730836'), ('eL4CeoiAa3lsAkERVsE_7Yyh4m5VIIsBgz0bR8HYol-ry-zDYYwNRKCqZ0JKnBiOlnImd2UXfcEXDw', 'NA1_3739607226')]
rate limit exceeded
waiting 2 minutes
8031 [('Q5n6W4N91n3Hny7qEtfvagf8X9lrsTfwCXSRTZZvQnQruuc3mGqPd4362V-qCqXwfhE2o74bXs-jdg', 'NA1_3548379652'), ('Q5n6W4N91n3Hny7qEtfvagf8X9lrsTfwCXSRTZZvQnQruuc3mGqPd4362V-qCqXwfhE2o74bXs-jdg', 'NA1_3548391531')]
rate limit exceeded
waiting 2 minutes
10665 [('ppuEFRdv26V1-YVxKx8PTZ3u-57eSRRzTBlfWkkIYmqZrp1C8kqtQysql70tGliCOVqYXstDY695Vw', 'NA1_3534814320'), ('ppuEFRdv26V1-YVxKx8PTZ3u-57eSRRzTBlfWkkIYmqZrp1C8kqtQysql70tGliCOVqYXstDY695Vw', 'NA1_3474632347')]
rate limit exceeded
waiting 2 minutes
13163 [

94699 [('ibvTY14o-aT29D3f79Tj5KNOkuByCicCO9PVgaCuyFJSX6PKxQbt1_7JtJwkdch7ceh2eW33bJLzug', 'NA1_3751134252'), ('ibvTY14o-aT29D3f79Tj5KNOkuByCicCO9PVgaCuyFJSX6PKxQbt1_7JtJwkdch7ceh2eW33bJLzug', 'NA1_3750938943')]
rate limit exceeded
waiting 2 minutes
97639 [('4p6TqcT1bdtAtZt-oCfIv4k4i5tWUycQSfgxu4VYNtBZMyG6NHug3TxGHlKy6AqgD9zltibICFwJ0A', 'NA1_3723360291'), ('4p6TqcT1bdtAtZt-oCfIv4k4i5tWUycQSfgxu4VYNtBZMyG6NHug3TxGHlKy6AqgD9zltibICFwJ0A', 'NA1_3722894736')]
rate limit exceeded
waiting 2 minutes
100489 [('gUZ-xVUZ07flRdmXeSc7nrbDGZKEcznr8RE80cGu9YwALR6UOJ2_74Qu3n5Wu9FVHi_UvVsCWO1rXQ', 'NA1_3733597734'), ('gUZ-xVUZ07flRdmXeSc7nrbDGZKEcznr8RE80cGu9YwALR6UOJ2_74Qu3n5Wu9FVHi_UvVsCWO1rXQ', 'NA1_3733613101')]
rate limit exceeded
waiting 2 minutes
103369 [('f6wqdLCUs_FpEutGlsj0vPTeOqxaGJsjZVf-Ib5fN1VRmGlGb8MKWVy9mDRrg4glPwfELn7q49lfHA', 'NA1_3756651500'), ('f6wqdLCUs_FpEutGlsj0vPTeOqxaGJsjZVf-Ib5fN1VRmGlGb8MKWVy9mDRrg4glPwfELn7q49lfHA', 'NA1_3756429126')]
rate limit exceeded
waiting 2 minutes
10

In [38]:
query6 = 'INSERT INTO matchids VALUES (%s,%s)'

In [41]:
adding_to_db(username, password, query6, match_histories)

In [31]:
match = "NA1_3749485149"
response = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/{}?api_key={}".format(match, key))

In [32]:
response.json()

{'metadata': {'data_version': '5',
  'match_id': 'NA1_3749485149',
  'participants': ['kGrLcumpNtF5wUKilJdPqhrcdIfU0Vnrn2d_03SA3EFzxSEdjzK5Qi-G3n01_vpn3CaDj5XdBjpOVA',
   'vY5sP4sFLeVdFXAHtI7fXRIlrPoUBbQBT07Na7osbXThcr3v-n-1bWwWRD81u_ipvGFq3Yq1lDL_eQ',
   'VprqLW1K1VVw0mJDzhOyd8YyC4Bx3n0wyu8Apa22RSG5abw-zjHcXRryDzyvZyGqvVrkcexHmQ5h2Q',
   'zAMcfLeA8mczep80FMzDIzTXLwipBd9Q7tQTFdeaCV9YwP0wnwBC9CDPzjLFh8YQG6klzeCsG33kXA',
   '0XYUnvqRVWJBnrIhkGvFftdj-k0zmlATrP5JD6Xb9vPXzHPUIGbW6-pmpe2wOYIZ_6my9gxKW1zIlA',
   'dlJtYZ5Rh4qGE7cyMFi9x-KhebqJkzXQn_Ts7xFJzS_akRm2dW5cFDNSDxFDRj5NxbMHZHLmZN2Ywg',
   'M0Gjt-oY2fKpqFKTrT9NuNgYpU80GkykhJYdBBXxnjm3GvGmIDUPs8kgy3C4fiL18Kf_DXHixtMF1g',
   'rhvZgOGclIFbtDGIJrbQMy5crvQSzVLvgB0L10VyRpTdf--SSVZ4AzTiKqOfACoWTBc_O_E0kmRQcg']},
 'info': {'game_datetime': 1611004367842,
  'game_length': 1973.95849609375,
  'game_version': 'Version 11.1.352.5559 (Jan 07 2021/11:01:16) [PUBLIC] <Releases/11.1>',
  'participants': [{'companion': {'content_ID': 'c159fe8e-a7c6-4256

In [35]:
match_data = []
for player in response.json()['info']['participants']:
                game_datetime = response.json()['info']['game_datetime']
                game_length = response.json()['info']['game_length']
                game_version = response.json()['info']['game_version']
                gold_left = player['gold_left']
                last_round = player['last_round']
                level = player['level']
                placement = player['placement']
                puuid = player['puuid']
                time_eliminated = player['time_eliminated']
                total_damage = player['total_damage_to_players']
                traits = player['traits']
                units = player['units']
                match_data.append(
                    (game_datetime, game_length, game_version, gold_left, last_round, level, placement,
                    puuid, time_eliminated, total_damage, traits, units))

In [36]:
match_data

[(1611004367842,
  1973.95849609375,
  'Version 11.1.352.5559 (Jan 07 2021/11:01:16) [PUBLIC] <Releases/11.1>',
  95,
  17,
  5,
  7,
  'kGrLcumpNtF5wUKilJdPqhrcdIfU0Vnrn2d_03SA3EFzxSEdjzK5Qi-G3n01_vpn3CaDj5XdBjpOVA',
  892.15283203125,
  0,
  [{'name': 'Cultist',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 3},
   {'name': 'Fortune',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 2},
   {'name': 'Set4_Shade',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 3},
   {'name': 'Set4_Vanguard',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 4}],
  [{'character_id': 'TFT4_Evelynn',
    'items': [4],
    'name': '',
    'rarity': 2,
    'tier': 1},
   {'character_id': 'TFT4_Sejuani',
    'items': [3],
    'name': '',
    'rarity': 3,
    'tier': 1}]),
 (1611004367842,
  1973.95849609375,
  'Version 11.1.352.5559 (Jan 07 2021/11:01:16) [PUBLIC] <Releases/11.1>',
  41,
  13,


In [21]:
# A lot of data so going to call the matchids in batches
def match_data_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT DISTINCT(match) FROM matchids WHERE match NOT IN (SELECT match FROM matches) LIMIT 1000")
    matches = c.fetchall()
    # Getting a list of all puuids to loop through
    games = [match[0] for match in matches]
    match_data = []
    for game in games:
        response = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/{}?api_key={}".format(game, key))
        start_time = time.time()
        if response.status_code == 200:
            for player in response.json()['info']['participants']:
                game_id = game
                game_datetime = response.json()['info']['game_datetime']
                game_length = response.json()['info']['game_length']
                game_version = response.json()['info']['game_version']
                gold_left = player['gold_left']
                last_round = player['last_round']
                level = player['level']
                placement = player['placement']
                puuid = player['puuid']
                time_eliminated = player['time_eliminated']
                total_damage = player['total_damage_to_players']
                traits = json.dumps(player['traits'])
                units = json.dumps(player['units'])
                match_data.append(
                    (game_id, game_datetime, game_length, game_version, gold_left, last_round, level, placement,
                    puuid, time_eliminated, total_damage, traits, units))
                end_time = time.time()
                
                if end_time - start_time < 1:
                    time.sleep(1 - (end_time - start_time))
                
        elif response.status_code == 429:
            print(match_data[-1])
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return match_data;

In [22]:
matches = match_data_collector(key, username, password)

('NA1_3740199018', 1610409114040, 2370.032470703125, 'Version 11.1.352.5559 (Jan 07 2021/11:01:16) [PUBLIC] <Releases/11.1>', 0, 40, 8, 3, 'HrLKA7Q-TbLHxr84tMCse1DAH8NkYw_cTLgbQdr1Wr45Um4D_Rr7W1YXRxcGSZ0xyy4Y01G4Q3DMhA', 2289.062255859375, 184, '[{"name": "Duelist", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 4}, {"name": "Dusk", "num_units": 2, "style": 1, "tier_current": 1, "tier_total": 3}, {"name": "Emperor", "num_units": 1, "style": 3, "tier_current": 1, "tier_total": 1}, {"name": "Fortune", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 2}, {"name": "Keeper", "num_units": 3, "style": 1, "tier_current": 1, "tier_total": 3}, {"name": "Set4_Assassin", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 3}, {"name": "Set4_Brawler", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 4}, {"name": "Set4_Vanguard", "num_units": 1, "style": 0, "tier_current": 0, "tier_total": 4}, {"name": "Sharpshooter", "num_units": 1, "style": 0, "tier_c

In [24]:
query9 = 'INSERT INTO matches VALUES (%s,%s, %s,%s,%s,%s,%s, %s, %s, %s, %s, %s, %s)'

In [25]:
adding_to_db(username, password, query9, matches)

In [23]:
len(matches)

7712